In [9]:
using Flux
using MLDatasets

In [13]:
train_x_raw, train_y_raw = MNIST(split = :train)[:]
println(typeof(train_x_raw))
println(size(train_x_raw))
test_x_raw, test_y_raw  = MNIST(split = :test)[:]
train_x = Flux.flatten(train_x_raw)
println(typeof(train_x))
println(size(train_x))
test_x = Flux.flatten(test_x_raw)
train_y = Flux.onehotbatch(train_y_raw, 0:9)
println(typeof(train_y))
println(size(train_y))
test_y  = Flux.onehotbatch(test_y_raw, 0:9)
;

Array{Float32, 3}
(28, 28, 60000)
Matrix

{Float32}
(784, 60000)
OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}
(10, 60000)


In [11]:
model = Chain(
  Dense(784, 128, relu),
  Dense(128, 64, relu),
  Dense(64, 32, relu),
  Dense(32, 10),
  softmax
)

loss(x, y) = Flux.crossentropy(model(x), y)

for epoch in 1:100
  Flux.train!(loss, Flux.params(model), [(train_x, train_y)], ADAM())
end

model

Chain(
  Dense(784 => 128, relu),              # 100_480 parameters
  Dense(128 => 64, relu),               # 8_256 parameters
  Dense(64 => 32, relu),                # 2_080 parameters
  Dense(32 => 10),                      # 330 parameters
  NNlib.softmax,
)                   # Total: 8 arrays, 111_146 parameters, 434.664 KiB.

In [12]:
p = model(test_x)
a = vec(first.(Tuple.(Flux.argmax(p, dims=1)))) .- 1
sum(a .== test_y_raw) / length(test_y_raw)

0.8961